In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import date
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

# colors
color1 = '#024758'
color2 = '#c9fb37'
color3 = '#226a7a'


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )

colors = {
    'background': '#024758',
    'text': 'white'
}


# data
# *******************************************************************************
df = pd.read_csv('emissions.csv')
df.timestamp = pd.to_datetime(df.timestamp)

# cards
# ******************************************************************************

card_household = dbc.Card([
                            dbc.CardImg(src="/assets/house_icon.png",
                                        top=True, bottom=False),
                            dbc.CardBody([
                                html.H4(id='houseHold'),
                                html.H6("of an american household weekly energy consumption", className="card-title")])
                        ], color=color1, outline=False)

card_car = dbc.Card([
                            dbc.CardImg(src="/assets/car_icon.png",
                                        top=True, bottom=False),
                            dbc.CardBody([
                                html.H4(id='car'),
                                html.H6("miles driven", className="card-title")])
                        ], color=color1, outline=False)

card_tv = dbc.Card([
                            dbc.CardImg(src="/assets/tv_icon.png",
                                        top=True, bottom=False),
                            dbc.CardBody([
                                html.H4(id='tv'),
                                html.H6("of TV", className="card-title")])

                    ], color=color1, outline=False)


# Layout section: Bootstrap (https://hackerthemes.com/bootstrap-cheatsheet/)
# *******************************************************************************

app.layout = dbc.Container([
    dbc.Row([
        # holding logo, subtitle, date selector
        dbc.Col([
                html.Img(src='/assets/logo.png'),
                html.P('Track and reduce CO2 emissions from your computing'),
                dcc.DatePickerRange(id='periode',
                        day_size=39,
                        month_format='MMMM Y',
                        end_date_placeholder_text='MMMM Y',
                        # should be calculated from today() like minus 1 week
                        start_date=date(2020, 1, 1),
                        min_date_allowed=date(2000, 1, 1),
                        max_date_allowed=date.today(),
                        initial_visible_month=date.today(),
                        end_date=date.today())
        ], xs=12, sm=12, md=12, lg=4, xl=4),  # if small screen the col would take the full width
        # holding indicators graph
       dbc.Col(dcc.Graph(id='indicators'), xs=12, sm=12, md=12, lg=8, xl=8
       )
    ]),

    dbc.Row([
        # holding project selector
        dbc.Col(
                dcc.Checklist(id='projectChecklist',
                            options=[{'label': projectName, 'value': projectName}
                                for projectName in df.project_name.unique()],
                            value=[df.project_name.unique()[0]],
                            labelClassName="mr-3"
                             ), xs=12, sm=12, md=12, lg=4, xl=4


        ),
        # horlding pieCharts
        dbc.Col(
                dcc.Graph(id='pieCharts'), xs=12, sm=12, md=12, lg=8, xl=8
        )
    ]),

    dbc.Row([
        # holding cards

                    dbc.Col(card_household, width={"size": 2, "offset": 4}),
                    dbc.Col(card_car, width=2),
                    dbc.Col(card_tv, width=2)

                    ],justify="around",)

    ])





# callback section: connecting the components
# ************************************************************************

# indicators
# -------------------------------------------------------------------------
@ app.callback(
    Output(component_id='indicators', component_property='figure'),
    [Input(component_id='periode', component_property='start_date'),
    Input(component_id='periode', component_property='end_date')]
)

def update_indicator(start_date, end_date):

    dff=df.copy()
    dff=df[df['timestamp'] > start_date][df['timestamp'] < end_date]

    # graph
    figIndic=make_subplots(rows = 1, cols = 3, specs = [
                    [{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]])

    figIndic.add_trace(go.Indicator(value=dff.energy_consumed.sum(),
                         title = {
                             "text": "Energy Consumed<br><span style='font-size:0.8em;color:gray'>kWh</span>"},
                          ), row=1, col=1)
    figIndic.add_trace(go.Indicator(value = dff.emissions.sum(),
                           title = {
                               "text": "Emissions<br><span style='font-size:0.8em;color:gray'>kg eq.C02</span>"},
                          ), row=1, col=2)
    figIndic.add_trace(go.Indicator(value = round(dff.duration.sum()/60, 0),
                           title = {
                               "text": "Duration<br><span style='font-size:0.8em;color:gray'>min.</span>"},
                          ), 1, 3)

    figIndic.update_layout(
                    title_text="GLOBAL ",
                    title_font_color='#d8d8d8',

                    font=dict(color = 'white'),
                    height=250,
                    margin=dict(
                    l = 10,
                    r = 10,
                    b = 10,
                    t = 40,
                    pad = 4
                    ),
                    paper_bgcolor=color1,
                    showlegend=False,
                    )

    return figIndic


# pieCharts and cards
# -----------------------------------------------------------------------------------

@ app.callback(
    [Output(component_id = 'pieCharts', component_property = 'figure'),
    Output(component_id='houseHold', component_property='children'),
     Output(component_id='car', component_property='children'),
     Output(component_id='tv', component_property='children')
    ],
    [Input(component_id='periode', component_property='start_date'),
    Input(component_id='periode', component_property='end_date'),
    Input(component_id='projectChecklist', component_property='value')]
)

def update_pieCharts(start_date, end_date, projectList):

    dff=df.copy()
    dff=df[df['timestamp'] > start_date][df['timestamp'] < end_date]



    energyConsumed=dff[dff['project_name'].isin(
        projectList)].energy_consumed.sum()
    emission=dff[dff['project_name'].isin(projectList)].emissions.sum()
    duration=dff[dff['project_name'].isin(projectList)].duration.sum()

    houseHold=str(round(100*emission/160.58, 2)) + " %"
    car=str(round(emission / 0.409, ))
    time_in_minutes=(emission * (1 / 0.097) * 60)
    tvTime= str(time_in_minutes)+ " min"
    if time_in_minutes >= 60:
        time_in_hours = time_in_minutes / 60
        tvTime = "{:.0f} hours".format(time_in_hours)
        if time_in_hours >= 24:
            time_in_days = time_in_hours / 24
            tvTime = "{:.0f} days".format(time_in_days)


    string1=str(energyConsumed) + ' KWh'
    string2=str(emission) + ' kg eq.CO2'
    string3=str(round(duration/60,)) + ' min'


    figPie=make_subplots(rows = 1, cols = 3, specs = [
                        [{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]])
    figPie.add_trace(go.Pie(values=[energyConsumed, dff.energy_consumed.sum()-energyConsumed], name="energy consumed", textinfo='none', hole=.8, marker=dict(colors=[color2, color3]), title=''), row = 1, col = 1)
    figPie.add_trace(go.Pie(values=[emission, dff.emissions.sum()-emission], name="emission", textinfo='none', hole=.8, marker=dict(colors=[color2, color3])), row = 1, col = 2)
    figPie.add_trace(go.Pie(values=[duration, (dff.duration.sum()-duration)], name="duration", textinfo='none', hole=.8, marker=dict(colors=[color2, color3])), row= 1, col = 3)


    figPie.update_layout(
        title_text= "PER PROJECT ",
        title_font_color= '#d8d8d8',
        font= dict(color='white'),
        paper_bgcolor= color1,
        showlegend= False,
        annotations= [dict(text= string1, font=dict(color='white',), x=0.05, y=0.5, font_size=30, showarrow=False),dict(text=string2, x=0.5, y=0.5, font_size=20, showarrow=False),dict(text=string3,font=dict(color='white',), x=0.93, y=0.5, font_size=30, showarrow=False),],
        margin=dict(l=10, r=10,b=10,t=40,pad=4),
        height= 200)
    
   
    
    return figPie, houseHold, car, tvTime





if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [2]:
houseHold = 5
figHouseHold= go.Figure(go.Indicator(value= houseHold, title = {"text":"%"}))
    
figHouseHold.update_layout(paper_bgcolor=color1,font=dict(color='white'),)

NameError: name 'go' is not defined